In [1]:
using Pkg
cd("D:\\Home\\Git\\Oceananigans.jl")
# cd("C:\\Users\\Ali\\Documents\\Git\\Oceananigans.jl\\")
Pkg.activate(".");

In [2]:
using Statistics, Printf, BenchmarkTools
using FFTW
using Oceananigans, Oceananigans.Operators

In [3]:
struct SpectralSolverParameters{T<:AbstractArray}
    kx²::T
    ky²::T
    kz²::T
    FFT!
    DCT!
    IFFT!
    IDCT!
end


let pf2s = Dict(FFTW.ESTIMATE => "FFTW.ESTIMATE",
                FFTW.MEASURE => "FFTW.MEASURE",
                FFTW.PATIENT => "FFTW.PATIENT",
                FFTW.EXHAUSTIVE => "FFTW.EXHAUSTIVE")
    global plannerflag2string
    plannerflag2string(k::Integer) = pf2s[Int(k)]
end

function SpectralSolverParameters(g::Grid, exfield::CellField, planner_flag=FFTW.PATIENT)
    kx² = zeros(eltype(g), g.Nx)
    ky² = zeros(eltype(g), g.Ny)
    kz² = zeros(eltype(g), g.Nz)
    
    for i in 1:g.Nx; kx²[i] = (2sin((i-1)*π/g.Nx)    / (g.Lx/g.Nx))^2; end
    for j in 1:g.Ny; ky²[j] = (2sin((j-1)*π/g.Ny)    / (g.Ly/g.Ny))^2; end
    for k in 1:g.Nz; kz²[k] = (2sin((k-1)*π/(2g.Nz)) / (g.Lz/g.Nz))^2; end
    
    print("Planning Fourier transforms... (planner_flag=$(plannerflag2string(planner_flag)))\n")
    print("FFT!:  "); @time FFT!  = FFTW.plan_fft!(exfield.data, [1, 2]; flags=planner_flag)
    print("IFFT!: "); @time IFFT! = FFTW.plan_ifft!(exfield.data, [1, 2]; flags=planner_flag)
    print("DCT!:  "); @time DCT!  = FFTW.plan_r2r!(exfield.data, FFTW.REDFT10, 3; flags=planner_flag)
    print("IDCT!: "); @time IDCT! = FFTW.plan_r2r!(exfield.data, FFTW.REDFT01, 3; flags=planner_flag)
    
    SpectralSolverParameters{Array{eltype(g),1}}(kx², ky², kz², FFT!, DCT!, IFFT!, IDCT!)
end

SpectralSolverParameters

In [4]:
plannerflag2string(FFTW.PATIENT)

"FFTW.PATIENT"

In [5]:
function solve_poisson_3d_ppn_planned!(ssp::SpectralSolverParameters, g::RegularCartesianGrid, f::CellField, ϕ::CellField)
    ssp.DCT!*f.data  # Calculate DCTᶻ(f) in place.
    ssp.FFT!*f.data  # Calculate FFTˣʸ(f) in place.

    for k in 1:g.Nz, j in 1:g.Ny, i in 1:g.Nx
        @inbounds ϕ.data[i, j, k] = -f.data[i, j, k] / (ssp.kx²[i] + ssp.ky²[j] + ssp.kz²[k])
    end
    ϕ.data[1, 1, 1] = 0

    ssp.IFFT!*ϕ.data  # Calculate IFFTˣʸ(ϕ) in place.
    ssp.IDCT!*ϕ.data  # Calculate IDCTᶻ(ϕ) in place.
    @. ϕ.data = ϕ.data / (2*g.Nz)
    nothing
end

solve_poisson_3d_ppn_planned! (generic function with 1 method)

In [ ]:
N = (100, 100, 100)
L = (2000, 2000, 2000)
g = RegularCartesianGrid(N, L; dim=3, FloatType=Float64)
tmp = TemporaryFields(g)

A = rand(Complex{Float64}, g.Nx, g.Ny, g.Nz)
@. A = real(A)

B1 = solve_poisson_3d_ppn(real(A), g.Nx, g.Ny, g.Nz, g.Δx, g.Δy, g.Δz);

tmp.fCC1.data .= A
tmp.fCC2.data .= 0
solve_poisson_3d_ppn!(g, tmp.fCC1, tmp.fCC2)
B2 = copy(tmp.fCC2.data)

tmp.fCC1.data .= A
tmp.fCC2.data .= 0
ssp = SpectralSolverParameters(g, tmp.fCC1, FFTW.MEASURE);
tmp.fCC1.data .= A
tmp.fCC2.data .= 0
solve_poisson_3d_ppn_planned!(ssp, g, tmp.fCC1, tmp.fCC2)
B3 = copy(tmp.fCC2.data)
@. B3 = real(B3)

@show B1 ≈ B2
@show B2 ≈ B3
@show B1 ≈ B3;

In [6]:
N = (256, 256, 256)
L = (2000, 2000, 2000)

c = EarthConstants()
eos = LinearEquationOfState()

g = RegularCartesianGrid(N, L; dim=3, FloatType=Float64)

U  = VelocityFields(g)
tr = TracerFields(g)
pr = PressureFields(g)
G  = SourceTerms(g)
Gp = SourceTerms(g)
F  = ForcingFields(g)
tmp = TemporaryFields(g)

U.u.data  .= 0
U.v.data  .= 0
U.w.data  .= 0
tr.S.data .= 35
tr.T.data .= 283;

In [7]:
@benchmark solve_poisson_3d_ppn($tr.T.data, $g.Nx, $g.Ny, $g.Nz, $g.Δx, $g.Δy, $g.Δz)

BenchmarkTools.Trial: 
  memory estimate:  3.75 GiB
  allocs estimate:  83886346
  --------------
  minimum time:     5.038 s (5.26% GC)
  median time:      5.038 s (5.26% GC)
  mean time:        5.038 s (5.26% GC)
  maximum time:     5.038 s (5.26% GC)
  --------------
  samples:          1
  evals/sample:     1

In [8]:
@benchmark solve_poisson_3d_ppn!($g, $tmp.fCC1, $tmp.fCC2)

BenchmarkTools.Trial: 
  memory estimate:  20.94 KiB
  allocs estimate:  219
  --------------
  minimum time:     2.696 s (0.00% GC)
  median time:      2.732 s (0.00% GC)
  mean time:        2.732 s (0.00% GC)
  maximum time:     2.767 s (0.00% GC)
  --------------
  samples:          2
  evals/sample:     1

In [9]:
tmp.fCC1.data .= rand(eltype(g), g.Nx, g.Ny, g.Nz)
ssp = SpectralSolverParameters(g, tmp.fCC1, FFTW.PATIENT);

Planning Fourier transforms... (planner_flag=FFTW.PATIENT)
FFT!:    2.475688 seconds (46 allocations: 3.063 KiB)
IFFT!:   2.376198 seconds (53 allocations: 3.438 KiB)
DCT!:    8.411680 seconds (59 allocations: 4.047 KiB)
IDCT!:   8.888404 seconds (59 allocations: 4.047 KiB)


In [10]:
tmp.fCC1.data .= rand(eltype(g), g.Nx, g.Ny, g.Nz)
@benchmark solve_poisson_3d_ppn_planned!(ssp, g, tmp.fCC1, tmp.fCC2)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     882.176 ms (0.00% GC)
  median time:      892.932 ms (0.00% GC)
  mean time:        892.067 ms (0.00% GC)
  maximum time:     900.105 ms (0.00% GC)
  --------------
  samples:          6
  evals/sample:     1

In [ ]:
function IFFTFFT!(A)
    FFTW.fft!(A)
    FFTW.ifft!(A)
    nothing
end

function IFFTFFT_planned(A, FFT!, IFFT!)
    IFFT!*(FFT!*A)
    nothing
end

In [ ]:
A = rand(Complex{Float64}, 100, 100, 100)
@. A = real.(A);
@benchmark ifft(fft(A))

In [ ]:
A = rand(Complex{Float64}, 100, 100, 100)
@. A = real.(A);
@benchmark IFFTFFT!(A)

In [ ]:
@time FFT!  = FFTW.plan_fft!(A; flags=FFTW.PATIENT);
@time IFFT! = FFTW.plan_ifft!(A; flags=FFTW.PATIENT);
# @time P = FFTW.plan_fft!(A; flags=FFTW.EXHAUSTIVE);
# @time P⁻¹ = FFTW.plan_ifft!(A; flags=FFTW.EXHAUSTIVE);

In [ ]:
A = rand(Complex{Float64}, 100, 100, 100)
@. A = real.(A);
@show A[10, 10, 10]
@time FFT!*A
@show A[10, 10, 10]
@time IFFT!*A
@show A[10, 10, 10];

In [ ]:
@benchmark IFFTFFT_planned($A, $FFT!, $IFFT!)

In [ ]:
A = rand(Complex{Float64}, 100, 100, 100);
@time FFT!  = FFTW.plan_fft!(A; flags=FFTW.PATIENT);
@time IFFT! = FFTW.plan_ifft!(A; flags=FFTW.PATIENT);

B = zeros(Float64, 100, 100, 100)
@time DCT!  = FFTW.plan_r2r!(B, FFTW.REDFT10, 3; flags=FFTW.PATIENT)
@time IDCT! = FFTW.plan_r2r!(B, FFTW.REDFT01, 3; flags=FFTW.PATIENT)

In [ ]:
@show typeof(FFT!)
@show typeof(IFFT!)
@show typeof(DCT!)
@show typeof(IDCT!)

In [ ]:
IFFT!